## 기본 세팅

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tqdm import tqdm
from glob import glob

import cv2
import warnings
import torch
import json

In [3]:
%matplotlib inline

## 이미지 crop 및 df 생성

In [4]:
# 기본 path 설정
path = '/content/drive/MyDrive/Data/PJT/'

In [5]:
# 새로 저장할 폴더 path 설정
new_path = path + '대용량플라스틱통/'

In [6]:
# 사진 폴더 및 json 폴더 경로
folder_list = sorted(glob(path+'img/대용량플라스틱통/*')) # 사진
label_folder_list = sorted(glob(path+'json/대용량플라스틱통/*')) # json

# 길이가 맞아야 합니다.. 정상적인 업로드가 됬다면 맞을 수 밖에 없음
len(folder_list), len(label_folder_list) 

(479, 479)

In [7]:
# df 제작을 위한 배열 생성
name_list = []
class_list = []
detail_list = []
damage_list = []
trans_list = []

# 처음 데이터를 만들때 polygon 형태로 라벨링이 된 경우는 일단 제외
# 폴리곤 형태면 (x1,y1) (x2,y2)로 받을 수 없고 리스트 형태로 좌표를 받아야함..
poly_list = []

# 새로 지정될 사진의 이름을 설정 ex) cup, bottle, basket ...
name = 'big_plastic'
count = 0

for folder, label_folder in tqdm(zip(folder_list, label_folder_list)):
  file_list = sorted(glob(folder+'/*'))
  label_file_list = sorted(glob(label_folder+'/*'))

  # 이미지 파일 read 및
  for f, label in zip(file_list, label_file_list):
    # json file read
    with open(label, 'r') as ff:
      json_data = json.load(ff)
    
    d = json_data['Bounding'][0]
    
    # image data
    if d['Drawing'] == 'BOX':
      new_name = name + str(count)
      class_ = d['CLASS']
      detail = d['DETAILS']
      damage = d['DAMAGE']
      trans = d['TRANSPARENCY']
      x1 = int(d['x1'])
      x2 = int(d['x2'])
      y1 = int(d['y1'])
      y2 = int(d['y2'])

      # img crop
      img = cv2.imread(f)
      img = img[y1:y2, x1:x2]

      # 새로 지정된 이름으로 저장
      cv2.imwrite(new_path + new_name + '.jpg', img)

      # 배열에 저장
      name_list.append(new_name)
      class_list.append(class_)
      detail_list.append(detail)
      damage_list.append(damage)
      trans_list.append(trans)

      count += 1
    else:
      poly_list.append(f.split('/')[-1])

479it [12:14,  1.53s/it]


In [8]:
# 제외된 사진 갯수
len(poly_list)

35

In [9]:
# dataframe 제작
df = pd.DataFrame({'name':name_list,
                   'class':class_list,
                   'detail':detail_list,
                   'damage':damage_list,
                   'trans':trans_list})

In [10]:
df.tail()

,name,class,detail,damage,trans
1234,big_plastic1234,플라스틱류,대용량플라스틱통,원형,불투명
1235,big_plastic1235,플라스틱류,대용량플라스틱통,원형,불투명
1236,big_plastic1236,플라스틱류,대용량플라스틱통,원형,불투명
1237,big_plastic1237,플라스틱류,대용량플라스틱통,원형,불투명
1238,big_plastic1238,플라스틱류,대용량플라스틱통,원형,불투명


In [11]:
path

'/content/drive/MyDrive/Data/PJT/'

In [12]:
df.to_csv(path+'big_plastic.csv', index=False)